In [4]:
# Just imports
from astropy.constants import sigma_sb, k_B
from math import pi, log
from astropy import units

In [3]:
# Temperature with infinite time in free space
T0 = 3.3*units.K
# Set variables for problem statement
# any of these may be adjusted in further iterations of the problem
M = 3.285*(10**23)*units.kg
Q = 3.846*(10**26)*units.W
V = 198*units.m/units.s
T_sun = 5778 * units.K
# Properties of steel
steel_rho = 8000*units.kg/(units.m)**3  # density
steel_c = 420*units.J/(units.kg*units.K)  # specific heat
steel_k = 45*units.W/(units.m*units.K)  # thermal condutivity, not using at the moment

### Figure of Merit
We will define the quantity we wish to optimize here, which are the number of computations per second
that our hypothetical construction enables given all the parameters.
These parameters are not solved for at the moment, so they are taken as inputs.

$$
\bar{\dot{C}} = \frac{2}{\pi} \frac{ \dot{Q} }{ T_H k_B \ln{2} } \left( \frac{ T_H }{ T_{c2} - T_{c1}} \ln{\frac{T_{c2}}{T_{c1}}} -1 \right)
$$

We'll define that, and then try to do a few test calculations.

In [14]:
def FOM(Tc1, Tc2):
    return ((2/pi)*Q/(T_sun*k_B*log(2))*(T_sun/(Tc2-Tc1)*log(Tc2/Tc1)-1)).decompose()

In [15]:
FOM(500*units.K, 250*units.K)

<Quantity 6.65079504e+46 1 / s>

In [18]:
FOM(50*units.K, 25*units.K)

<Quantity 7.04931092e+47 1 / s>

In [17]:
FOM(394*units.K, 393.9*units.K)

<Quantity 6.05161239e+46 1 / s>

Already, we have something to compare to `Dyson_Sphere_Reference.ipynb`.
That reported 7e46 Hz at 394 K, which is awfully close.
They should really be the actual same, and I could let the theorist in me stress about this, but... later.

### Supporting Equations
These are mostly trivial equations, but they help to formalize here,
becase we need everything in terms of $R$, so we can have these handy in terms of that variable.

$$
d = \frac{ M }{ 2 \pi^2 R^2 \rho }
$$

$$
a = \frac{ \sigma }{ d \rho c }
$$

In [21]:
def d(R):
    return (M/(2*pi**2*R**2*steel_rho)).decompose()

In [24]:
def a(R):
    return (sigma_sb/(d(R)*steel_rho*steel_c)).decompose()

### Energy Balance
This balance equation states that the radiator must be removing as much heat as the engine is producing.

$$
\dot{Q} = \rho d \pi R V c \left( T_{c2} - T_{c1} \right)
$$

In practice, this will be reported as $\Delta T$

In [26]:
def delta_t(R):
    return (Q/(steel_rho*d(R)*pi*R*V*steel_c)).decompose()

In [32]:
delta_t(units.AU/100000.)

<Quantity 132331.94804545 K>

In [34]:
(units.AU/100000.).to(units.km)

<Quantity 1495.978707 km>

In [35]:
M

<Quantity 3.285e+23 kg>

In [36]:
delta_t(1000.*units.m)

<Quantity 88.45844358 K>

In [37]:
d(1000.*units.m)

<Quantity 2.08025055e+12 m>

In [38]:
(Q/(M*steel_c)).decompose()

<Quantity 2.78756251 K / s>

### Intermission

I've ran into a problem when looking at the actual numbers.
Put simply, the value selected for $M$, the mass of Mercury, is completely insufficient for the heat load selected.
That heat load is selected to be the output of the sun.
You can see that by the above simple division, that the heat output of the sun would heat up the _entire_ mass of Mercury (as steel) by 3 Kelvin per second.
That necessarily means that you would have to run that entire planetary mass through the heat exchanger in a very short amount of time, which also makes the selected velocity of 200 m/s horribly naive, leaving the equations here far out of their domain of validity even in the best of circumstances.